# Last API with pylast

[Oficial Website](https://www.last.fm/api/)

[PyLast Repo](https://github.com/pylast)

In [1]:
import pandas as pd 
import numpy as np 
import pylast

import os
import json
import sys
import requests
import time 

sys.path.append('../scripts/') 

from create_last_database import User
from create_last_database import Track
from create_last_database import Artist
from create_last_database import Album
from create_last_database import Tag

## Network with the API through PyLast

In [2]:
API_KEY = input()
API_SECRET = input()

network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET)

Let's get some random users built by `generate_lastfm_users.py`

In [3]:
def get_random_users(filepath: str, quantity: int = 1000, random_state: int = 200) -> pd.DataFrame:
    
    users = pd.read_csv(filepath)
    chosen_users = users.sample(n = quantity, replace = False, random_state = random_state, axis = 'index')
    chosen_users.index = list(range(0,len(chosen_users)))

    return chosen_users

user_path = "../data/lastfm-api/users_lastfm.csv"

users = get_random_users(user_path)
users.head()

,user_id,user_name
0,22330,elleciel
1,20893,Wolfet93
2,22057,drewogg
3,15821,Sharenge
4,14704,old_urach


Now we have 1k users, we can build the database for them with the main aspects

In [10]:
users_class = User(network, user_path)
if not os.path.exists('../data/lastfm-api/1k_users_info_lastfm.json'):
    with open('../data/lastfm-api/1k_users_info_lastfm.json', 'w') as f:
        json.dump({}, f)
for user_id, user_name in users.as_matrix():
    with open('../data/lastfm-api/1k_users_info_lastfm.json', 'r+') as f:
        data = json.load(f)
        if str(user_id) in data:
            continue
        user_info = users_class.get_user_info(user_name)
        data[user_id] = user_info
        f.seek(0)
        json.dump(data, f)
        if len(data) % 100 == 0:
            print(len(data))